### Install & import dependencies

In [1]:
%pip install ndlib --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep

### Load data

In [3]:
path = "../data/loc-brightkite_edges.txt.gz"
G = nx.read_edgelist(path, create_using=nx.Graph(), nodetype=int)

### Threshold
Setup

In [4]:
model = ep.ThresholdModel(G)

config = mc.Configuration()
# How many nodes to infect initially
# you can also set individual nodes to be infected by adding a node configuration
# See https://ndlib.readthedocs.io/en/latest/reference/models/epidemics/Threshold.html#parameters
config.add_model_parameter('fraction_infected', 0.1)
threshold = 0.25
for n in G.nodes():
    config.add_node_configuration("threshold", n, threshold)
model.set_initial_status(config)

Simulation

In [5]:
iterations = model.iteration_bunch(200)

In [11]:
for it in iterations:
    i = it['iteration']
    num_infected = sum(it['status'].values())
    print(f"Iteration {i}: {num_infected} newly infected nodes ({num_infected/len(G)*100:.1f}%)")

Iteration 0: 5822 newly infected nodes (10.0%)
Iteration 1: 7505 newly infected nodes (12.9%)
Iteration 2: 4342 newly infected nodes (7.5%)
Iteration 3: 5324 newly infected nodes (9.1%)
Iteration 4: 7825 newly infected nodes (13.4%)
Iteration 5: 11883 newly infected nodes (20.4%)
Iteration 6: 9619 newly infected nodes (16.5%)
Iteration 7: 3418 newly infected nodes (5.9%)
Iteration 8: 765 newly infected nodes (1.3%)
Iteration 9: 173 newly infected nodes (0.3%)
Iteration 10: 49 newly infected nodes (0.1%)
Iteration 11: 17 newly infected nodes (0.0%)
Iteration 12: 4 newly infected nodes (0.0%)
Iteration 13: 0 newly infected nodes (0.0%)
Iteration 14: 0 newly infected nodes (0.0%)
Iteration 15: 0 newly infected nodes (0.0%)
Iteration 16: 0 newly infected nodes (0.0%)
Iteration 17: 0 newly infected nodes (0.0%)
Iteration 18: 0 newly infected nodes (0.0%)
Iteration 19: 0 newly infected nodes (0.0%)
Iteration 20: 0 newly infected nodes (0.0%)
Iteration 21: 0 newly infected nodes (0.0%)
Iterat